In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import re

In [ ]:
# Prepare the environment
!apt update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,064 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col, explode, udf, isnan, when, count, col, avg
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import StringType, IntegerType, DoubleType
from pyspark.ml.feature import StringIndexer, Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/My Drive/MDS0/LDS0_K287_Online_MaQuocDung/data/'

Mounted at /content/gdrive
/content/gdrive/My Drive/MDS0/LDS0_K287_Online_MaQuocDung/data


In [ ]:

# Create a SparkContext
sc = SparkContext()

# Create a Spark session with custom configuration
spark = SparkSession(sc).builder \
    .appName("MyApp") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

## Bước 1: Business Understanding

Dựa vào yêu cầu nói trên => xác định vấn đề:

Chưa có hệ thống Recommendation System


=> Mục tiêu/ vấn đề: Xây dựng Recommendation System cho một hoặc một số nhóm hàng hóa trên tiki.vn giúp đề xuất và gợi ý cho người dùng/ khách hàng. => Xây dựng các mô hình đề xuất:


*   Content-based filtering
*   Collaborative filtering

## Bước 2: Data Understanding

In [ ]:
data = spark.read.csv("Sendo_reviews_preprocessing.csv", header=True, inferSchema=True)

In [ ]:
data.show()

+--------------------+------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             content|rating|sentiment|         new_content|neutral_words_count|negative_words_count|positive_words_count|negation_words_count|positive_emoji_count|negative_emoji_count|
+--------------------+------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Shop phục vụ khá ...|     4| positive|   shop phục_vụ tốt |                  0|                   0|                   1|                   0|                   0|                   0|
|Sản phẩm gần giốn...|     3|  neutral|     sản_phẩm mô_tả |                  0|                   0|                   0|                   0|                   0|                   0|
|Giao hàng nhanh b...|     5| positive|          giao hàng |          

### Thực hiện EDA

## Bước 3: Data Preparation/Prepare

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, NaiveBayes
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.classification import GBTClassifier, LinearSVC

import numpy as np

In [ ]:
# Chuyển đổi cột 'sentiment' thành dạng số
sentiment_indexer = StringIndexer(inputCol="sentiment", outputCol="label")
data = sentiment_indexer.fit(data).transform(data)

In [ ]:
# Tách văn bản thành các từ
tokenizer = Tokenizer(inputCol="content", outputCol="words")
data = tokenizer.transform(data)

In [ ]:
# Loại bỏ các từ dừng cho cột 'content'
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
data = stop_words_remover.transform(data)

In [ ]:
# Sử dụng TF-IDF để biểu diễn cột 'content'
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(hashingTF.transform(data))
data = idf_model.transform(hashingTF.transform(data))

In [ ]:
# Chuyển đổi các cột có kiểu dữ liệu chuỗi thành kiểu dữ liệu số
string_columns = ["positive_words_count", "neutral_words_count", "negative_words_count", "positive_emoji_count", "negative_emoji_count"]

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index").fit(data) for col in string_columns]
data = data

for indexer in indexers:
    data = indexer.transform(data)

In [ ]:
data.show()

+--------------------+------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------------+-------------------------+--------------------------+--------------------------+--------------------------+
|             content|rating|sentiment|         new_content|neutral_words_count|negative_words_count|positive_words_count|negation_words_count|positive_emoji_count|negative_emoji_count|label|               words|      filtered_words|        raw_features|            features|positive_words_count_index|neutral_words_count_index|negative_words_count_index|positive_emoji_count_index|negative_emoji_count_index|
+--------------------+------+---------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------

In [ ]:
# Danh sách các cột số sau khi chuyển đổi
numeric_columns = ["positive_words_count_index", "neutral_words_count_index", "negative_words_count_index", "positive_emoji_count_index", "negative_emoji_count_index"]  # Thay thế bằng tên các cột đã được chuyển đổi

In [ ]:
# Sử dụng VectorAssembler để kết hợp các cột số
assembler = VectorAssembler(inputCols=["features"] + numeric_columns, outputCol="assembled_features")
data = assembler.transform(data)

In [ ]:
data.select("assembled_features").show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------+
|assembled_features                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------+
|(10005,[1471,3341,4328,4683,8427,10000],[2.2309587053327222,4.498119790127632,2.805025796621481,1.9163444608321178,1.7903964398209444,1.0])|
+-------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [ ]:
# Chọn cột nhãn và cột đặc trưng để sử dụng
df = data.select('assembled_features', 'label')

In [ ]:
df.show()

+--------------------+-----+
|  assembled_features|label|
+--------------------+-----+
|(10005,[1471,3341...|  0.0|
|(10005,[2904,5993...|  2.0|
|(10005,[4898,5384...|  0.0|
|(10005,[1167,1471...|  0.0|
|(10005,[29,1141,1...|  0.0|
|(10005,[29,1141,1...|  0.0|
|(10005,[2904,4972...|  0.0|
|(10005,[1615,2904...|  0.0|
|(10005,[3058,4849...|  0.0|
|(10005,[874,1202,...|  0.0|
|(10005,[698,1202,...|  0.0|
|(10005,[1167,1471...|  0.0|
|(10005,[2904,4972...|  0.0|
|(10005,[2904,4972...|  0.0|
|(10005,[1615,2904...|  0.0|
|(10005,[2904,4972...|  0.0|
|(10005,[4171,6177...|  0.0|
|(10005,[2904,4972...|  0.0|
|(10005,[1167,1280...|  0.0|
|(10005,[29,1141,1...|  0.0|
+--------------------+-----+
only showing top 20 rows



## Bước 4&5: Modeling & Evaluation/Analyze

In [ ]:
# Split the data into training and testing sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Danh sách các bộ phân loại bạn muốn đánh giá
classifiers = [
    ("Logistic Regression", LogisticRegression(labelCol="label", featuresCol="assembled_features", maxIter=10)),
    ("Decision Tree", DecisionTreeClassifier(labelCol="label", featuresCol="assembled_features")),
    ("Random Forest", RandomForestClassifier(labelCol="label", featuresCol="assembled_features")),
    ("Naive Bayes", NaiveBayes(labelCol="label", featuresCol="assembled_features"))
]

In [ ]:
# Define a list to store the results
results = []

# Define the feature columns (assuming 'assembled_features' contains the feature vectors)
feature_columns = ["assembled_features"]

In [ ]:
# Loop through each classifier
for name, classifier in classifiers:
    # Train the classifier on the training data
    model = classifier.fit(train_data)

    # Make predictions on the test data
    predictions = model.transform(test_data)

    # Evaluate the classifier
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)

    # Calculate F1 Score for each class
    metrics = MulticlassMetrics(predictions.select("prediction", "label").rdd)
    f1_positive = metrics.fMeasure(0.0)
    f1_neutral = metrics.fMeasure(1.0)
    f1_negative = metrics.fMeasure(2.0)

    # Calculate confusion matrix for each class
    confusion_matrix = metrics.confusionMatrix().toArray()

    # Calculate ROC AUC for Positive class
    #predictions = model.transform(test_data)
    #evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    #roc_auc_positive = evaluator.evaluate(predictions.filter(predictions.label == 1.0))

    # Calculate CCC (Concordance Correlation Coefficient) for each class
    y_true_positive = predictions.filter(predictions.label == 1.0).select("label").rdd.flatMap(lambda x: x).collect()
    y_pred_positive = predictions.filter(predictions.label == 1.0).select("prediction").rdd.flatMap(lambda x: x).collect()
    ccc_positive = np.corrcoef(y_true_positive, y_pred_positive)[0, 1]

    # Calculate recall and precision for each class
    recall_positive = metrics.recall(0.0)
    recall_neutral = metrics.recall(1.0)
    recall_negative = metrics.recall(2.0)

    precision_positive = metrics.precision(0.0)
    precision_neutral = metrics.precision(1.0)
    precision_negative = metrics.precision(2.0)

    results.append({
        "Model": name,
        "Accuracy": accuracy,
        "F1 Score (Positive)": f1_positive,
        "F1 Score (Neutral)": f1_neutral,
        "F1 Score (Negative)": f1_negative,
        "Recall (Positive)": recall_positive,
        "Recall (Neutral)": recall_neutral,
        "Recall (Negative)": recall_negative,
        "Precision (Positive)": precision_positive,
        "Precision (Neutral)": precision_neutral,
        "Precision (Negative)": precision_negative,
        "Confusion Matrix": confusion_matrix.tolist(),
        #"ROC AUC (Positive)": roc_auc_positive,
        #"CCC (Positive)": ccc_positive
    })

Py4JJavaError: ignored

In [ ]:
# Display the results
for result in results:
    print(result)

In [ ]:
from tabulate import tabulate

In [ ]:
# Tạo DataFrame từ results
df = pd.DataFrame(results)

# Hiển thị bảng
print(tabulate(df, headers='keys', tablefmt='pretty'))

## Bước 6: Deployment and Feedback/Act

Phần CCC bị lỗi chưa chạy ra được kết quả kỳ vọng

Mô hình Random Forest không dự đoán được các trường hợp neutral và negative

Logistic Regression: Mô hình có độ chính xác (Accuracy) khá cao (khoảng 90.3%), nhưng F1 Score của lớp negative khá thấp. Mô hình có khả năng tốt trong việc phân loại các lớp positive và neutral, nhưng gặp khó khăn khi phân loại lớp negative.

Decision Tree: Mô hình Decision Tree có độ chính xác tương đối cao (khoảng 92.5%). F1 Score của các lớp đều khá cao, đặc biệt là lớp positive có F1 Score lên đến 0.96. Mô hình có độ nhạy cao đối với lớp positive.

Random Forest: Mô hình Random Forest có độ chính xác tương đối (khoảng 89.7%). Mô hình hoạt động tốt với lớp positive nhưng không có khả năng phân loại các lớp khác.

Naive Bayes: Mô hình Naive Bayes có độ chính xác tương đối (khoảng 89.9%). Mô hình có khả năng tốt trong việc phân loại lớp positive và neutral, nhưng gặp khó khăn trong việc phân loại lớp negative, tuy nhiên đây là mô hình cho kết quả neutral và negative tốt nhất.

Dựa trên đánh giá trên, do dữ liệu bị mất cân bằng nên sẽ ưu tiên nhận diện 2 phân lớp neutral và negative, nên quyết định lựa chọn Naive Bayes

Sau khi cho thêm dữ liệu vào chạy trên bigdata thì bị tràn ram nên sử dụng số liệu phân tích cũ